# Preliminaries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

#sys.path.append("/Users/paolo/Documents/methods/CMI_FS")
#from feature_selection import forwardFeatureSelection

sys.path.append("../LinCFA")
from LinCFA import LinCFA

sys.path.append("../NonLinCFA")
from NonLinCFA import NonLinCFA

sys.path.append("../GenLinCFA")
from GenLinCFA import GenLinCFA

sys.path.append("../droughts")
from aux import prepare_target,prepare_features,compare_methods

#from aux import standardize,unfold_dataset,compute_r2,prepare_target,prepare_features,aggregate_unfolded_data,aggregate_unfolded_data_onlyTrain,FS_with_linearWrapper,compare_methods, compute_r2


# Data

In [2]:
### Load and standardize target
target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path='../droughts/Emiliani1.csv')


target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.379890    0.50  2001     1 -0.382765
1    2001-01-13  0.482679    0.58  2001     2  0.319215
2    2001-01-21  0.516259    0.59  2001     3  0.548542
3    2001-01-29  0.434421    0.50  2001     5 -0.010351
4    2001-02-06  0.494805    0.54  2001     6  0.402030
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.427085    0.43  2009    48 -0.060454
407  2009-12-05  0.547380    0.57  2009    49  0.761079
408  2009-12-13  0.531070    0.58  2009    50  0.649694
409  2009-12-21  0.295704    0.00  2009    52 -0.957702
410  2009-12-29  0.027861    0.00  2009    53 -2.786888

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))


In [3]:
### Load and standardize features
variables_list = ['cyclostationary_mean_tg', 
                 'cyclostationary_mean_tg_1w',
                 'cyclostationary_mean_tg_4w', 
                 'cyclostationary_mean_tg_8w',
                 'cyclostationary_mean_tg_12w', 
                 'cyclostationary_mean_tg_16w',
                 'cyclostationary_mean_tg_24w',
                 'cyclostationary_mean_rr', 
                 'cyclostationary_mean_rr_1w',
                 'cyclostationary_mean_rr_4w', 
                 'cyclostationary_mean_rr_8w',
                 'cyclostationary_mean_rr_12w', 
                 'cyclostationary_mean_rr_16w',
                 'cyclostationary_mean_rr_24w'
                 ]

df_train = pd.DataFrame()
df_val = pd.DataFrame()
df_test = pd.DataFrame()
df_trainVal = pd.DataFrame()

for variable in variables_list:
    df_train_unfolded_std, df_val_unfolded_std, df_test_unfolded_std,df_trainVal_unfolded_std = prepare_features('../droughts/Emiliani1_aggreg.csv',variable,False,max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
    df_train_unfolded_std = df_train_unfolded_std.add_prefix(variable)
    df_val_unfolded_std = df_val_unfolded_std.add_prefix(variable)
    df_test_unfolded_std = df_test_unfolded_std.add_prefix(variable)
    df_trainVal_unfolded_std = df_trainVal_unfolded_std.add_prefix(variable)
    df_train = pd.concat((df_train,df_train_unfolded_std),axis=1)
    df_val = pd.concat((df_val,df_val_unfolded_std),axis=1)
    df_test = pd.concat((df_test,df_test_unfolded_std),axis=1)
    df_trainVal = pd.concat((df_trainVal,df_trainVal_unfolded_std),axis=1)
    
df_trainVal

,cyclostationary_mean_tgmean_12.149860342381333_43.74986055078544,cyclostationary_mean_tgmean_12.149860342381333_43.8498605504681,cyclostationary_mean_tgmean_12.149860342381333_43.94986055015075,cyclostationary_mean_tgmean_12.149860342381333_44.04986054983341,cyclostationary_mean_tgmean_12.149860342381333_44.14986054951606,cyclostationary_mean_tgmean_12.149860342381333_44.24986054919872,cyclostationary_mean_tgmean_12.149860342381333_44.34986054888138,cyclostationary_mean_tgmean_12.149860342381333_44.44986054856403,cyclostationary_mean_tgmean_12.149860342381333_44.54986054824669,cyclostationary_mean_tgmean_12.149860342381333_44.64986054792934,...,cyclostationary_mean_rr_24wmean_11.349860345581744_44.849860547294654,cyclostationary_mean_rr_24wmean_11.449860345181692_44.14986054951606,cyclostationary_mean_rr_24wmean_11.449860345181692_44.24986054919872,cyclostationary_mean_rr_24wmean_11.449860345181692_44.34986054888138,cyclostationary_mean_rr_24wmean_11.449860345181692_44.44986054856403,cyclostationary_mean_rr_24wmean_11.449860345181692_44.54986054824669,cyclostationary_mean_rr_24wmean_11.449860345181692_44.64986054792934,cyclostationary_mean_rr_24wmean_11.449860345181692_44.749860547612,cyclostationary_mean_rr_24wmean_11.449860345181692_44.849860547294654,cyclostationary_mean_rr_24wmean_10.749860347982052_44.24986054919872
0,0.719982,0.737633,0.791074,0.848973,0.983768,1.172439,1.201792,1.127842,0.986823,0.794499,...,2.331970,6.168345,6.215139,5.406085,4.240808,4.540304,2.472063,2.105030,2.296531,3.363161
1,1.845931,2.162888,2.278888,2.313132,2.330406,2.363919,2.119857,2.008556,1.797622,1.626867,...,1.670322,3.527914,3.841691,3.108603,2.947959,2.824932,1.593425,1.462249,1.728906,3.004583
2,-0.312107,-0.404970,-0.361520,-0.116017,0.037568,0.199148,0.215971,0.233859,0.106281,-0.117314,...,1.118337,1.915294,2.358315,2.981883,2.858264,2.382361,1.067353,1.006560,1.055914,1.950734
3,1.666516,1.661740,1.601745,1.595421,1.579804,1.588420,1.391649,1.185715,1.141724,0.968893,...,0.534584,2.429799,2.789340,2.202089,1.991279,1.546552,0.466434,0.489244,0.506180,2.157730
4,-0.031846,-0.003213,-0.013352,0.122002,0.295776,0.473944,0.532393,0.421560,0.238369,-0.025356,...,1.056396,2.718182,3.000691,3.082463,3.061446,2.457115,1.281718,1.138486,1.142762,2.350811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,0.578850,0.609135,0.689113,0.967141,1.273710,1.237462,1.007356,1.032989,1.194737,1.011196,...,0.690343,1.041006,0.388026,0.123181,0.347596,0.465708,-0.240666,0.449174,0.650788,1.164679
635,1.243632,1.043466,0.783567,0.922940,1.124030,1.209410,1.205085,1.367888,1.900526,1.924692,...,0.666434,0.955184,0.294397,0.090659,0.410255,0.581733,-0.169828,0.462829,0.652467,1.096017
636,0.267909,0.598843,0.778573,1.011130,1.340522,1.491446,1.432183,1.526572,1.965525,1.990127,...,0.894777,1.047647,0.425352,0.262704,0.623916,0.826442,0.059008,0.692457,0.885935,1.242053
637,0.957833,1.053095,1.249241,1.296098,1.319729,1.135229,0.976383,1.250418,1.632408,1.738173,...,0.878993,1.036936,0.456154,0.304824,0.500595,0.733759,0.026283,0.689393,0.807281,1.119456


In [4]:
### Together
df_trainVal_withTar = pd.concat((df_trainVal,target_df_trainVal['mean_std']), axis=1)
df_trainVal_withTar

,cyclostationary_mean_tgmean_12.149860342381333_43.74986055078544,cyclostationary_mean_tgmean_12.149860342381333_43.8498605504681,cyclostationary_mean_tgmean_12.149860342381333_43.94986055015075,cyclostationary_mean_tgmean_12.149860342381333_44.04986054983341,cyclostationary_mean_tgmean_12.149860342381333_44.14986054951606,cyclostationary_mean_tgmean_12.149860342381333_44.24986054919872,cyclostationary_mean_tgmean_12.149860342381333_44.34986054888138,cyclostationary_mean_tgmean_12.149860342381333_44.44986054856403,cyclostationary_mean_tgmean_12.149860342381333_44.54986054824669,cyclostationary_mean_tgmean_12.149860342381333_44.64986054792934,...,cyclostationary_mean_rr_24wmean_11.449860345181692_44.14986054951606,cyclostationary_mean_rr_24wmean_11.449860345181692_44.24986054919872,cyclostationary_mean_rr_24wmean_11.449860345181692_44.34986054888138,cyclostationary_mean_rr_24wmean_11.449860345181692_44.44986054856403,cyclostationary_mean_rr_24wmean_11.449860345181692_44.54986054824669,cyclostationary_mean_rr_24wmean_11.449860345181692_44.64986054792934,cyclostationary_mean_rr_24wmean_11.449860345181692_44.749860547612,cyclostationary_mean_rr_24wmean_11.449860345181692_44.849860547294654,cyclostationary_mean_rr_24wmean_10.749860347982052_44.24986054919872,mean_std
0,0.719982,0.737633,0.791074,0.848973,0.983768,1.172439,1.201792,1.127842,0.986823,0.794499,...,6.168345,6.215139,5.406085,4.240808,4.540304,2.472063,2.105030,2.296531,3.363161,-0.382765
1,1.845931,2.162888,2.278888,2.313132,2.330406,2.363919,2.119857,2.008556,1.797622,1.626867,...,3.527914,3.841691,3.108603,2.947959,2.824932,1.593425,1.462249,1.728906,3.004583,0.319215
2,-0.312107,-0.404970,-0.361520,-0.116017,0.037568,0.199148,0.215971,0.233859,0.106281,-0.117314,...,1.915294,2.358315,2.981883,2.858264,2.382361,1.067353,1.006560,1.055914,1.950734,0.548542
3,1.666516,1.661740,1.601745,1.595421,1.579804,1.588420,1.391649,1.185715,1.141724,0.968893,...,2.429799,2.789340,2.202089,1.991279,1.546552,0.466434,0.489244,0.506180,2.157730,-0.010351
4,-0.031846,-0.003213,-0.013352,0.122002,0.295776,0.473944,0.532393,0.421560,0.238369,-0.025356,...,2.718182,3.000691,3.082463,3.061446,2.457115,1.281718,1.138486,1.142762,2.350811,0.402030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,0.578850,0.609135,0.689113,0.967141,1.273710,1.237462,1.007356,1.032989,1.194737,1.011196,...,1.041006,0.388026,0.123181,0.347596,0.465708,-0.240666,0.449174,0.650788,1.164679,-0.320516
635,1.243632,1.043466,0.783567,0.922940,1.124030,1.209410,1.205085,1.367888,1.900526,1.924692,...,0.955184,0.294397,0.090659,0.410255,0.581733,-0.169828,0.462829,0.652467,1.096017,0.137056
636,0.267909,0.598843,0.778573,1.011130,1.340522,1.491446,1.432183,1.526572,1.965525,1.990127,...,1.047647,0.425352,0.262704,0.623916,0.826442,0.059008,0.692457,0.885935,1.242053,0.305368
637,0.957833,1.053095,1.249241,1.296098,1.319729,1.135229,0.976383,1.250418,1.632408,1.738173,...,1.036936,0.456154,0.304824,0.500595,0.733759,0.026283,0.689393,0.807281,1.119456,-0.586199


# NonLinCFA

In [14]:
l = []
for i in [0.01,0.001,0.0001,0.00001,0.000001]:
    for j in [0,1,2,3,4]:
        l.append([i,j])
for eps,seed in l:
    print(eps,seed)

0.01 0
0.01 1
0.01 2
0.01 3
0.01 4
0.001 0
0.001 1
0.001 2
0.001 3
0.001 4
0.0001 0
0.0001 1
0.0001 2
0.0001 3
0.0001 4
1e-05 0
1e-05 1
1e-05 2
1e-05 3
1e-05 4
1e-06 0
1e-06 1
1e-06 2
1e-06 3
1e-06 4


In [5]:
for eps in [0.01,0.001,0.0001,0.00001,0.000001]:
    for curr_seed in [0,1,2,3]:
        curr_df_trainVal = df_trainVal[np.random.default_rng(seed=curr_seed).permutation(df_trainVal.columns.values)]
        curr_df_test = df_test[np.random.default_rng(seed=curr_seed).permutation(df_test.columns.values)]
        curr_df_trainVal_withTar = pd.concat((curr_df_trainVal,target_df_trainVal['mean_std']), axis=1)
        
        output = NonLinCFA(curr_df_trainVal_withTar,'mean_std', eps, -5 , 0).compute_clusters()
        
        aggregate_trainVal = pd.DataFrame()
        aggregate_test = pd.DataFrame()
        for i in range(len(output)):
            aggregate_trainVal[str(i)] = curr_df_trainVal_withTar[output[i]].mean(axis=1)
            aggregate_trainVal = aggregate_trainVal.copy()
            aggregate_test[str(i)] = curr_df_test[output[i]].mean(axis=1)
            aggregate_test = aggregate_test.copy()
        print(f'Number of aggregated features: {len(output)}\n')
        compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, list(aggregate_trainVal.columns))


674
1130
433
142
7
14
Number of aggregated features: 7

Full aggregate regression train score: 0.3541921766345436, test score: 0.3185643694838668
Aggregate regression train score with FS: 0.3541921766345436, test score: 0.3185643694838668
800
390
497
343
349
6
2
19
Number of aggregated features: 9

Full aggregate regression train score: 0.3747681682198791, test score: 0.2769713509124213
Aggregate regression train score with FS: 0.3747681682198791, test score: 0.2769713509124213
816
517
644
405
11
13
Number of aggregated features: 7

Full aggregate regression train score: 0.36854187792806703, test score: 0.28050717626784727
Aggregate regression train score with FS: 0.36854187792806703, test score: 0.28050717626784727
1097
547
399
354
9
Number of aggregated features: 6

Full aggregate regression train score: 0.36479974966424156, test score: 0.284424091830241
Aggregate regression train score with FS: 0.36479974966424156, test score: 0.284424091830241
591
915
170
345
314
18
24
15
1
8
4
2
1

KeyboardInterrupt: 

# GenLinCFA

In [12]:
#for variable in ['cyclostationary_mean_tg']:#variables_list:
#actual_df_trainVal = df_trainVal_unfolded_std[df_trainVal_unfolded_std.columns[pd.Series(df_trainVal_unfolded_std.columns).str.startswith(variable)]]
for eps in [0.45,0.455,0.46,0.465]:
    for curr_seed in [0,1,2,3]:
        curr_df_trainVal = df_trainVal[np.random.default_rng(seed=curr_seed).permutation(df_trainVal.columns.values)]
        curr_df_test = df_test[np.random.default_rng(seed=curr_seed).permutation(df_test.columns.values)]
        curr_df_trainVal_withTar = pd.concat((curr_df_trainVal,target_df_trainVal['mean_std']), axis=1)
        
        output = GenLinCFA(curr_df_trainVal_withTar,'mean_std', eps, -5 , 0, 1).compute_clusters()
        
        aggregate_trainVal = pd.DataFrame()
        aggregate_test = pd.DataFrame()
        for i in range(len(output)):
            aggregate_trainVal[str(i)] = curr_df_trainVal_withTar[output[i]].mean(axis=1)
            aggregate_trainVal = aggregate_trainVal.copy()
            aggregate_test[str(i)] = curr_df_test[output[i]].mean(axis=1)
            aggregate_test = aggregate_test.copy()
        print(f'Number of aggregated features: {len(output)}\n')
        compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, list(aggregate_trainVal.columns))


Number of aggregated features: 7

Full aggregate regression train score: 0.3419645417141758, test score: 0.2734277423665955
Aggregate regression train score with FS: 0.3419645417141758, test score: 0.2734277423665955
Number of aggregated features: 6

Full aggregate regression train score: 0.34770244087100066, test score: 0.29004806013374373
Aggregate regression train score with FS: 0.34770244087100066, test score: 0.29004806013374373
Number of aggregated features: 9

Full aggregate regression train score: 0.35276158568859106, test score: 0.28546968251694593
Aggregate regression train score with FS: 0.35276158568859106, test score: 0.28546968251694593
Number of aggregated features: 6

Full aggregate regression train score: 0.34325568020302477, test score: 0.28828584371562394
Aggregate regression train score with FS: 0.34325568020302477, test score: 0.28828584371562394
Number of aggregated features: 4

Full aggregate regression train score: 0.32878601231391014, test score: 0.264842127790

# LinCFA